In [1]:
#Kütüphaneleri yükleme
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [4]:

#Veri yükleme ve temizleme
df = pd.read_csv("/content/chess_data.csv")
print(df.head())
df = df[df['status'] != 'noStart']
df = df.dropna(subset=['white_rating', 'black_rating', 'winner', 'pgn', 'time_control'])
df = df.sample(n=25000, random_state=42)  # Az veri ile test

    game_id              white  white_rating    black  black_rating winner  \
0  Zqp04DJA       venajalainen        2743.0     gsvc        2909.0  black   
1  wpraROCc            ysakdzh        2080.0     gsvc        2928.0  white   
2  qVVp5qbc  bullettrainto2950        2790.0     gsvc        3048.0    NaN   
3  FmfVMgPR               gsvc        3047.0  oleg946        2622.0  white   
4  8Jo7FiwG        rain_chesss        2448.0     gsvc        3047.0  black   

    status  time_control  incriment  \
0   resign         180.0        2.0   
1  noStart         300.0        0.0   
2     draw          60.0        0.0   
3     mate          60.0        0.0   
4   resign          60.0        0.0   

                                                 pgn  
0  e4 c6 d4 d5 e5 c5 dxc5 e6 Qg4 Nc6 Nf3 h5 Qf4 B...  
1                                                 d4  
2  d4 d5 Nf3 e6 g3 c5 Bg2 cxd4 O-O Nf6 Nxd4 e5 Nf...  
3  b3 Nf6 Bb2 e6 Nf3 g6 g3 Bg7 Bg2 O-O d4 d5 O-O ...  
4  d4 d5 c4 e6 cxd5 e

In [5]:
#Açılış tipi çıkar (ilk 2 hamle)
def get_opening(pgn):
    moves = pgn.strip().split()
    return " ".join(moves[:2]) if len(moves) >= 2 else moves[0] if moves else "Unknown"

df.loc[:, 'Opening'] = df['pgn'].apply(get_opening)

In [6]:

#Girdi ve hedef
features = ['white_rating', 'black_rating', 'time_control', 'Opening']
target = 'winner'

X = df[features].copy()
y = df[target]

In [8]:
#Label Encoding
le_time = LabelEncoder()
X.loc[:, 'time_control'] = le_time.fit_transform(X['time_control'].astype(str))

le_opening = LabelEncoder()
X.loc[:, 'Opening'] = le_opening.fit_transform(X['Opening'])

le_y = LabelEncoder()
y = le_y.fit_transform(y)

In [9]:
#Eğitim ve test seti
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Ölçekleme (SVM ve Logistic Regression için)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
#Modelleri tanımla
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Logistic Regression": LogisticRegression(max_iter=5000),
    "Naive Bayes": GaussianNB(),
    "SVM": SVC(),
    "Linear Regression": LinearRegression()
}

#Tahmin ve performans
for name, model in models.items():
    if name in ["Logistic Regression", "SVM"]:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

    if name == "Linear Regression":
        y_pred = y_pred.round().astype(int)
        y_pred = y_pred.clip(0, len(le_y.classes_)-1)

    acc = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {acc:.4f}")


Random Forest Accuracy: 0.7154
Decision Tree Accuracy: 0.6512
KNN Accuracy: 0.7084
Logistic Regression Accuracy: 0.7384
Naive Bayes Accuracy: 0.6916
SVM Accuracy: 0.7342
Linear Regression Accuracy: 0.7372


# Yeni Bölüm